<a href="https://colab.research.google.com/github/architb1703/Toxic_Span/blob/Abhay/Finalprediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# First do the preprocessing on the test dataset......


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
###PATHS

test_path ='drive/MyDrive/Final/test.csv'  #will be used to open test.csv

model_path='drive/MyDrive/Final/semi_sup-6.pt'  # will be used to load model

save_path = 'drive/MyDrive/Final/predicted1.csv' # predicted.csv will store the predicted tags

In [ ]:
pip install contractions

     |████████████████████████████████| 245kB 8.4MB/s 
     |████████████████████████████████| 317kB 9.2MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.0-cp36-cp36m-linux_x86_64.whl size=81700 sha256=53963fa8828eb478e469927c095ce4cc61247c49e4c61c840754c491a7b75458
  Stored in directory: /root/.cache/pip/wheels/0a/90/61/87a55f5b459792fbb2b7ba6b31721b06ff5cf6bde541b40994
Successfully built pyahocorasick


In [ ]:
import sys
import numpy as np
import pandas as pd

In [ ]:
# import data
from ast import literal_eval
pred = pd.read_csv(test_path)
pred.head()

,text
0,"I completely agree, Sylvia is a moron."
1,"If Kitty is giving them $17,000 a year to sit ..."
2,If someone does not know that drug use can kee...
3,Maybe we can get those people to hand feed the...
4,the work of soros' whores


In [ ]:
# corpus = dup of "train"
corpus = pred.copy()


  

In [ ]:
# stats of words and chars
length_of_words = corpus['text'].apply(lambda x: len(x.split()))
length_of_text = corpus['text'].apply(lambda x: len(x))

In [ ]:

from nltk.tokenize import TreebankWordTokenizer as twt

def tokens(string):
    tokens = twt().tokenize(string)
    return tokens 

def spans_of_tokens(string):
    list_of_spans = list(twt().span_tokenize(string))
    return list_of_spans

corpus["tokenize"] = corpus["text"].apply(lambda x: tokens(x.replace('"', "'")))
corpus["spans_of_tokens"] = corpus["text"].apply(lambda x: spans_of_tokens(x.replace('"', "'")))

In [ ]:
corpus.head()

,text,tokenize,spans_of_tokens
0,"I completely agree, Sylvia is a moron.","[I, completely, agree, ,, Sylvia, is, a, moron...","[(0, 1), (2, 12), (13, 18), (18, 19), (20, 26)..."
1,"If Kitty is giving them $17,000 a year to sit ...","[If, Kitty, is, giving, them, $, 17,000, a, ye...","[(0, 2), (3, 8), (9, 11), (12, 18), (19, 23), ..."
2,If someone does not know that drug use can kee...,"[If, someone, does, not, know, that, drug, use...","[(0, 2), (3, 10), (11, 15), (16, 19), (20, 24)..."
3,Maybe we can get those people to hand feed the...,"[Maybe, we, can, get, those, people, to, hand,...","[(0, 5), (6, 8), (9, 12), (13, 16), (17, 22), ..."
4,the work of soros' whores,"[the, work, of, soros, ', whores]","[(0, 3), (4, 8), (9, 11), (12, 17), (17, 18), ..."


In [ ]:
# zipped together -> token, their spans
temp = []
for i in range(len(corpus)):
    temp.append(list(zip(corpus.tokenize[i], corpus.spans_of_tokens[i])))

for i in range(len(temp)):    
    temp[i] = [list(ele) for ele in temp[i]]  

corpus["zipped"] = temp
# del temp


In [ ]:
corpus.zipped[0]

[['I', (0, 1)],
 ['completely', (2, 12)],
 ['agree', (13, 18)],
 [',', (18, 19)],
 ['Sylvia', (20, 26)],
 ['is', (27, 29)],
 ['a', (30, 31)],
 ['moron', (32, 37)],
 ['.', (37, 38)]]

In [ ]:
#################### cleaning functions ####################
# emoji removing function
def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)

#puntuation remover
# from string import punctuation
# def strip_punctuation(text):
#     return ''.join(c for c in text if c not in punctuation)

#convert accented text
import unicodedata
def convert_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

#expand contraction
import contractions
import re 

# function to expand contractions
def expand_contractions(text):
    return(contractions.fix(text))
# function to remove digits
def remove_nums(text):
    return re.sub('[0-9]+', '', text)

# remove special chars
def remove_spe_chars(text):
    return re.sub('\.+', '.', text)
#################### cleaning functions ####################    

In [ ]:
#cleaning tokens (in zipped)
def clean_string(string):
    # string = string.lower()#lowercasing
    string = expand_contractions(string) #expand contraction
    string = remove_nums(string)#remove digits
    #numbers into words(could be done if required)# not require though
    string = convert_accented_chars(string)#convert accented
    string = deEmojify(string)#remove emoji
    string = remove_spe_chars(string)#combine all multiple full stops occurring together
    
    return string

def cleaning(iterable):
    for i in range(len(iterable)):
        iterable[i][0] = clean_string(iterable[i][0])    
    
    return iterable


In [ ]:
#remove empty tokens (in zipped)
def remove_empty(iterable):
    iterable = list(filter(lambda x: x[0] != '', iterable))
    return iterable

In [ ]:
# clean
corpus.zipped = corpus.zipped.apply(lambda x: cleaning(x))
# remove nulls
corpus.zipped = corpus.zipped.apply(lambda x: remove_empty(x))

In [ ]:
corpus.zipped[0]

[['I', (0, 1)],
 ['completely', (2, 12)],
 ['agree', (13, 18)],
 [',', (18, 19)],
 ['Sylvia', (20, 26)],
 ['is', (27, 29)],
 ['a', (30, 31)],
 ['moron', (32, 37)],
 ['.', (37, 38)]]

In [ ]:


def redeem_spans(lis):
    arr = []
    for item in lis:
        arr.append(item[1])
    return arr

def reddem_token(lis):
    arr = []
    for item in lis:
        arr.append(item[0])
    return arr

In [ ]:

corpus['span_final'] = corpus.zipped.apply(lambda x : redeem_spans(x))

corpus['token_final'] = corpus.zipped.apply(lambda x : reddem_token(x))

In [ ]:
corpus

,text,tokenize,spans_of_tokens,zipped,span_final,token_final
0,"I completely agree, Sylvia is a moron.","[I, completely, agree, ,, Sylvia, is, a, moron...","[(0, 1), (2, 12), (13, 18), (18, 19), (20, 26)...","[[I, (0, 1)], [completely, (2, 12)], [agree, (...","[(0, 1), (2, 12), (13, 18), (18, 19), (20, 26)...","[I, completely, agree, ,, Sylvia, is, a, moron..."
1,"If Kitty is giving them $17,000 a year to sit ...","[If, Kitty, is, giving, them, $, 17,000, a, ye...","[(0, 2), (3, 8), (9, 11), (12, 18), (19, 23), ...","[[If, (0, 2)], [Kitty, (3, 8)], [is, (9, 11)],...","[(0, 2), (3, 8), (9, 11), (12, 18), (19, 23), ...","[If, Kitty, is, giving, them, $, ,, a, year, t..."
2,If someone does not know that drug use can kee...,"[If, someone, does, not, know, that, drug, use...","[(0, 2), (3, 10), (11, 15), (16, 19), (20, 24)...","[[If, (0, 2)], [someone, (3, 10)], [does, (11,...","[(0, 2), (3, 10), (11, 15), (16, 19), (20, 24)...","[If, someone, does, not, know, that, drug, use..."
3,Maybe we can get those people to hand feed the...,"[Maybe, we, can, get, those, people, to, hand,...","[(0, 5), (6, 8), (9, 12), (13, 16), (17, 22), ...","[[Maybe, (0, 5)], [we, (6, 8)], [can, (9, 12)]...","[(0, 5), (6, 8), (9, 12), (13, 16), (17, 22), ...","[Maybe, we, can, get, those, people, to, hand,..."
4,the work of soros' whores,"[the, work, of, soros, ', whores]","[(0, 3), (4, 8), (9, 11), (12, 17), (17, 18), ...","[[the, (0, 3)], [work, (4, 8)], [of, (9, 11)],...","[(0, 3), (4, 8), (9, 11), (12, 17), (17, 18), ...","[the, work, of, soros, ', whores]"
...,...,...,...,...,...,...
789,"True.....for once UR right.\nBut the thing is,...","[True, ..., ..for, once, UR, right., But, the,...","[(0, 4), (4, 7), (7, 12), (13, 17), (18, 20), ...","[[True, (0, 4)], [., (4, 7)], [.for, (7, 12)],...","[(0, 4), (4, 7), (7, 12), (13, 17), (18, 20), ...","[True, ., .for, once, UR, right., But, the, th..."
790,Naw. The best way to remove the terrorists' po...,"[Naw., The, best, way, to, remove, the, terror...","[(0, 4), (5, 8), (9, 13), (14, 17), (18, 20), ...","[[Naw., (0, 4)], [The, (5, 8)], [best, (9, 13)...","[(0, 4), (5, 8), (9, 13), (14, 17), (18, 20), ...","[Naw., The, best, way, to, remove, the, terror..."
791,imagine if this dumb witch every cared about m...,"[imagine, if, this, dumb, witch, every, cared,...","[(0, 7), (8, 10), (11, 15), (16, 20), (21, 26)...","[[imagine, (0, 7)], [if, (8, 10)], [this, (11,...","[(0, 7), (8, 10), (11, 15), (16, 20), (21, 26)...","[imagine, if, this, dumb, witch, every, cared,..."
792,Why thank you for your vey civil remark. I ag...,"[Why, thank, you, for, your, vey, civil, remar...","[(0, 3), (4, 9), (10, 13), (14, 17), (18, 22),...","[[Why, (0, 3)], [thank, (4, 9)], [you, (10, 13...","[(0, 3), (4, 9), (10, 13), (14, 17), (18, 22),...","[Why, thank, you, for, your, vey, civil, remar..."


In [ ]:
final_data = corpus[['text', 'span_final', 'token_final']].copy()


In [ ]:
use_tokens=final_data['token_final'].tolist()
use_spans=final_data['span_final'].tolist()
use_text=final_data['text'].tolist()

# Load the model and predict

---



In [ ]:
#model1

!pip install transformers==2.6.0
!pip install seqeval
!pip install urllib3 --upgrade


#Importing libraries

import pandas as pd
import numpy as np
import pickle
from tqdm import tqdm, trange

import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torchtext import data
import transformers
from transformers import BertTokenizer, BertConfig, BertForTokenClassification, AdamW
from transformers import get_linear_schedule_with_warmup

from keras.preprocessing.sequence import pad_sequences
from seqeval.metrics import accuracy_score
from sklearn.metrics import f1_score, confusion_matrix

np.random.seed(42)
torch.manual_seed(12)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
 

     |████████████████████████████████| 542kB 8.0MB/s 
     |████████████████████████████████| 890kB 14.5MB/s 
     |████████████████████████████████| 133kB 34.6MB/s 
     |████████████████████████████████| 1.1MB 23.1MB/s 
     |████████████████████████████████| 3.7MB 44.0MB/s 
     |████████████████████████████████| 7.1MB 34.4MB/s 
     |████████████████████████████████| 71kB 10.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=200cab31eff90d44082d2c028a3f33f27dbfb1ba584db2af5beaa125a9c783cd
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
ERROR: botocore 1.19.36 has requirement urllib3<1.27,>=1.25.4; python_version != "3.4", but you'll have urllib3 1.24.3 which is incompatible.
     |████████████████████████████████| 51kB 4.5MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-cp36-none-any.whl size=16171 sha256=78b26afc49433ce992ec4bdcff29

/usr/local/lib/python3.6/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.2) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [ ]:
#Class to take in tokenizer and model along with input data and calculate the span average f1 score
class SpanAvgF1():
  def __init__(self, model, tokenizer, X, spans, maxlen=500):
    self.model = model
    self.model.cuda()
    self.tokenizer = tokenizer
    self.maxlen = maxlen
    self.X = X
    self.spans = spans
    
    self.prepare_data()

    self.X = torch.tensor(self.X)
    self.attention_mask = torch.tensor(self.attention_mask)

    data = TensorDataset(self.X, self.attention_mask)
    self.dataloader = DataLoader(data, batch_size=16, shuffle=False)
  
  def tokenize_data(self, x, s):
    sentence = []
    spans = []
    for i in range(len(x)):
      word = x[i]
      # label = y[i]
      tokenized_word = self.tokenizer.tokenize(word)
      sentence.extend(tokenized_word)
      curr = s[i][0]
      spans.append([curr, curr+len(tokenized_word[0])])
      curr += len(tokenized_word[0])
      for j in range(len(tokenized_word)-1):
        spans.append([curr, curr+len(tokenized_word[j+1])-2])
        curr += len(tokenized_word[j+1])-2
      spans[-1][-1] = s[i][1]

    return(sentence, spans)

  def get_attention_mask(self, x):
    return([[(i!=0) for i in text] for text in x])

  def prepare_data(self):
    for i in range(len(self.X)):
      self.X[i], self.spans[i] = self.tokenize_data(self.X[i], self.spans[i])
    self.X = pad_sequences([tokenizer.encode(text) for text in self.X], maxlen = self.maxlen, dtype='long', value=0.0, truncating='post', padding = 'post')
    # self.y = pad_sequences(self.y, maxlen=self.maxlen, value=2, dtype='long', truncating='post', padding='post')
    self.attention_mask = self.get_attention_mask(self.X)
  
  def get_text_lengths(self, masks):
    lengths = []
    for mask in masks:
      lengths.append(torch.sum(mask).item())
    return(lengths)

  def evaluate(self):
    predictions , true_labels = [], []
    for batch in self.dataloader:
      batch = tuple(t.to(device) for t in batch)
      b_input_id, b_input_mask = batch
      # b_input_id, b_input_mask, b_labels = batch
      self.model.eval()

      with torch.no_grad():
          outputs = self.model(b_input_id, token_type_ids=None, attention_mask=b_input_mask)
      logits = outputs[0].detach().cpu().numpy()
      predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
      true_labels.extend(b_input_mask)
    
    self.pred_tags = [[p_i for p_i, l_i in zip(p, l) if l_i != 0][1:-1] for p, l in zip(predictions, true_labels)]
    # else:
    #   self.pred_tags = predictions
    # self.pred_tags = predictions
    return self.pred_tags

In [ ]:
X = use_tokens.copy()
spans = use_spans.copy()


In [ ]:

modelorg = torch.load(model_path, map_location=torch.device('cpu'))
model= modelorg
tokenizer = BertTokenizer.from_pretrained('bert-base-cased', do_lower_case = False)

In [ ]:
model=modelorg
metric = SpanAvgF1(model, tokenizer, X, spans)
out=metric.evaluate()

In [ ]:
predicted_spans=[]
for i in range(len(spans)):
  s = [0 for k in range(len(use_text[i]))]
  prev = 0
  for j in range(len(spans[i])):
    for k in range(spans[i][j][0], spans[i][j][1]):
      s[k] = out[i][j]
    if(prev==1 and out[i][j]==1):
      for l in range(spans[i][j-1][1], spans[i][j][0]):
        s[l] = 1
    prev = out[i][j]
  predicted_spans.append(s)

In [ ]:
final_offsets=[]
for x in predicted_spans:
  lis=[]
  for i in range(len(x)):
    if x[i]==1:
      lis.append(i)
  final_offsets.append(lis)


In [ ]:

data = {'spans':final_offsets,'text':use_text}
 
df = pd.DataFrame(data)

df.to_csv(save_path,index=None)